In [3]:
import functools
import inspect

In [71]:
def compose(*workflow):
    def dec(loss):
        @functools.wraps(loss)
        def called(*args,**kwargs): # *args are for grad, **kwargs are the rest
            
            res = dict([])
            
            for f in workflow:
                sig = inspect.signature(f)
                f_args = sig.parameters.keys()
                
                feed_args = False
                feed_kwargs = False
                
                for arg in f_args:
                    if not feed_args or not feed_kwargs:
                        if arg in kwargs.keys() and arg not in res.keys():
                            feed_kwargs = True
                        elif arg not in kwargs.keys() and arg not in res.keys():
                            feed_args = True
                    else:
                        break

                if feed_args and feed_kwargs:
                    res = f(*args, **res, **kwargs)
                elif feed_args and not feed_kwargs:
                    res = f(*args, **res)
                elif not feed_args and feed_kwargs:
                    res = f(**res, **kwargs)
                else:
                    res = f(**res)
                        
            return loss(**res)
        
        return called
    
    return dec
            
  

In [72]:
import jax
import jax.numpy as jnp

In [83]:
def data_gen(p):
    return dict(data = jnp.array([3*p**2,4]))

def preprocess(params, data):
    s, b = data
    return dict(s = s + params, b = b)
    
@compose(data_gen, preprocess)
def loss(s,b):
    return s / b

jax.grad(loss)(55., params=1) # takes in args/kwargs specified in 'expose'

DeviceArray(82.5, dtype=float32)

make pipeline object, and do the arg stuff when calling?

could use decorator for appending to pipeline?